In [168]:
import numpy as np
import pandas as pd
import re
from scipy.sparse import lil_matrix
import scipy.spatial.distance 

In [169]:
#Open the files and store them as dataframes, data must be inserted into data file
dfMenu = pd.read_csv("Menu.csv")
dfMenuPage = pd.read_csv("MenuPage.csv")
dfMenuItem = pd.read_csv("MenuItem.csv")

In [161]:
menuDict = {} #create empty dict

for index, row in dfMenu.iterrows(): #iterate through the rows
    menuId = row[0] #save the Menu ID
    sponsor = row[2] #save the sponsor name
    
    #break if the length of the dictionary is 250
    if len(menuDict) == 250:
        break
    
    if re.search(r'\bDINNER\b', str(row[3])): #use regex to find when dinner is stated in the event row
        dishList = [] #create empty list to store dishes
        #find the ids(which is menu_page_id in MenuItem.csv) at the menuID for this individual menu
        seriesPageID = dfMenuPage.query(f'menu_id == {menuId}')['id']

        #for each menupage ID...
        for menuPageID in seriesPageID:
            seriesDishID = dfMenuItem.query(f'menu_page_id == {menuPageID}')['dish_id'].dropna() #...find the item IDs for each menuPageID and drop NaN values
            
            #find each dishID
            for dishID in seriesDishID:
                
                dishList.append(int(dishID)) #append the dish to the dish list

        #add the dish to the dictionary, making sure to append if it already exists
        if sponsor in menuDict:
            menuDict[sponsor] += dishList
        else:   
            menuDict[sponsor] = dishList

In [162]:
uniqueList = [] #create an empty list

#iterate through each list in the Menu Diction
for ilist in menuDict.values():
    
    #iterate through each dish in the list
    for dish in ilist:
        
        #if the dish is not in the unique list, add it
        if dish not in uniqueList:
            uniqueList.append(dish)
            
print(len(uniqueList))
print(uniqueList)

10091
[43, 44, 36635, 46, 47, 48, 49, 52, 55, 117, 58, 61, 132, 158, 177, 408, 83, 458, 797, 798, 800, 1253, 1258, 1264, 1267, 96, 1283, 112, 1286, 1287, 1289, 1291, 40533, 1281, 71, 72, 73, 54870, 130, 1025, 1027, 1030, 1031, 1055, 1035, 1036, 808, 80895, 1056, 54871, 1041, 1042, 1043, 1045, 1046, 1047, 41, 1049, 842, 1050, 1051, 1052, 1053, 1054, 124, 597, 601, 603, 607, 610, 614, 31493, 623, 20374, 71591, 275325, 275330, 859, 1097, 363782, 363783, 1261, 363784, 1277, 1279, 1280, 363785, 25636, 25638, 25640, 25641, 25646, 25648, 25651, 94171, 108911, 66036, 108912, 108913, 18162, 20335, 31501, 108914, 108915, 1248, 94175, 2571, 108916, 9173, 18410, 108917, 108918, 26775, 1129, 26782, 30419, 26787, 380, 26804, 10117, 26818, 26820, 5629, 26822, 26823, 54665, 3223, 28924, 35794, 23636, 23638, 269063, 22638, 23642, 9101, 35838, 6150, 23648, 23649, 23650, 35856, 163348, 163349, 163350, 1632, 35866, 6197, 35872, 35869, 33491, 9051, 1099, 35864, 34, 39, 135, 170, 245, 516, 518, 520, 527, 53

In [163]:
#create a dataframe with zeros, columns as the unique list and the index as the keys from the dictionary
df = pd.DataFrame(0, columns=uniqueList, index=(menuDict.keys()))

#iterate through each key and value pair in the dictionary
for sponsor, dishes in menuDict.items():
    
    #iterate through each dish in the list
    for dish in dishes:
        df.loc[sponsor, dish] += 1 #add 1 to that value in the dictionary

df.head(10)         
    

,43,44,36635,46,47,48,49,52,55,117,...,811,22628,43409,23853,43414,43403,43363,43513,23903,43407
REPUBLICAN HOUSE,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
NORDDEUTSCHER LLOYD BREMEN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CANADIAN PACIFIC RAILWAY COMPANY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MANHATTAN HOTEL,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
OCCIDENTAL & ORIENTAL STEAMSHIP COMPANY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NOVIOMAGUS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HOTEL SAVOY,2,0,0,0,1,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
HOTEL IMPERIAL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WILSON COLLEGE ALUMNAE ASSOCIATION,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
VETERANS AMERICAN GUARD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df_norm = df.divide(df.sum(axis=1), axis=0) #this performs L1 normalization
df_norm.head(10)

,43,44,36635,46,47,48,49,52,55,117,...,811,22628,43409,23853,43414,43403,43363,43513,23903,43407
REPUBLICAN HOUSE,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NORDDEUTSCHER LLOYD BREMEN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CANADIAN PACIFIC RAILWAY COMPANY,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MANHATTAN HOTEL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005682,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OCCIDENTAL & ORIENTAL STEAMSHIP COMPANY,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NOVIOMAGUS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOTEL SAVOY,0.011299,0.000000,0.000000,0.000000,0.005650,0.000000,0.000000,0.000000,0.000000,0.016949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOTEL IMPERIAL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WILSON COLLEGE ALUMNAE ASSOCIATION,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VETERANS AMERICAN GUARD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
def cos_sim(row):
    """
    This function takes in the name of the row and prints the top 10 most similar sponsors, utilizing cosine distance.
    Heavily inspired from Professor Cody Butain's code as seen below:
    https://github.com/cbuntain/umd.inst414/blob/main/Module03/02-Similarity.ActorsGenre.Normed.ipynb
    
    Inputs:
    row(string): A string representing the row 
    """

    #Gathering the genres for that sponsor
    target_sponsor = df.loc[row]

    #Generating distances from that sponsor to all the others
    distances = scipy.spatial.distance.cdist(df, [target_sponsor], metric="cosine")[:,0]

    query_distances = list(zip(df.index, distances))

    #Printing the top ten most similar sponsors to our target
    i=1
    for similar_sponosor, similar_dish_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
        print(f"{i}.", similar_sponosor, similar_dish_score, df.loc[similar_sponosor].sum())
        i+=1

In [165]:
cos_sim("CUNARD LINE")

1. CUNARD LINE 0.0 769
2. USMS 0.7818385713319268 303
3. USMS ST LOUIS 0.7955010172158054 86
4. U.S.M.S. 0.8041222783256909 196
5. HEADQUARTERS 47TH INFANTRY U.S. VOLUNTEERS 0.8090861996713197 18
6. U.S.M.S 0.8179078150346861 99
7. RED STAR LINE 0.8440014021961351 267
8. OCEAN STEAMSHIP CO. 0.8514036597666055 60
9. D&H DINING CAR SERVICE 0.8528092357366177 69
10. BATTERY PARK HOTEL 0.8621147685588016 260


In [166]:
cos_sim("NORDDEUTSCHER LLOYD BREMEN")

1. NORDDEUTSCHER LLOYD BREMEN 0.0 146
2. NORDDEUTSCHER LLOYDS BREMEN 0.6656330724547883 35
3. BREMEN NORDDEUTSCHER LLOYD 0.7911961834246675 34
4. HAMBURG-AMERIKA LINIE 0.850466564116232 28
5. ALPHA KAPPA PHI 0.8560690702733798 17
6. PENNSYLVANIA RAILROAD 0.8594196107211167 22
7. XIII CLUB 0.8637989550786003 15
8. WHITE STAR LINE 0.8637989550786003 15
9. MASONIC TEMPLE RESTAURANT 0.8638549703163885 19
10. NORDDEUTSCHER LLOYD -BREMEN 0.8643006180904298 34


In [172]:
cos_sim("OCCIDENTAL & ORIENTAL STEAMSHIP COMPANY")

1. OCCIDENTAL & ORIENTAL STEAMSHIP COMPANY 0.0 78
2. OCCIDENTAL & ORIENTAL STEAMSHIP CO. 0.6070212135217423 113
3. DEL CORONADO HOTEL 0.7257586221349277 50
4. MAXWELL HOUSE 0.7278344730240913 61
5. U.S. ARMY TRANSPORT 0.7334991045554868 31
6. PACIFIC MAIL STEAMSHIP COMPANY 0.7390439259906789 103
7. HOTEL ORMOND 0.7524631142558313 123
8. TAMPA BAY HOTEL 0.7549509852950983 98
9. TOYO KISEN KAISHA - HONG KONG MARU 0.758477054230176 35
10. HOTEL DEL CORONADO 0.7667152625920782 49
